# Workbook for optimizing foci segmentation thresholds

This workbook provides step-by-step instructions for determining segmentation thresholds for identifying foci within cells. __Do not edit this workbook file directly!__ Make a copy for each optimization that you perform and edit that version. Save this template in an unedited form for future experiments.

This workbook uses a built-in "optimization mode" that only pulls the first three images out of multi-image .czi files to dramatically speed up processing. This will allow you to tinker with thresholds and find the ideal cutoffs.

I found that the most effective way to define these cutoffs was to use a positive control image - one where there were many foci - and a negative control image with few to no foci to set the thresholds. This allows you to identify cutoffs that simultaneously maximize sensitivity while minimizing false positives.

__Before running this notebook, you must have completed the setup notebook in the install folder.__

In [6]:
# import required code
import sys
sys.path.append('/Users/nweir/Dropbox/code/csth-imaging')
from csth_analysis import find_cells, segment_cells, foci
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

<IPython.core.display.Javascript object>

The cell below defines the file paths for the files you will need during this process. Change each variable as needed.

In [1]:
neg_ctrl_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_dFIP200_NoTreat_10Pos_1_AiryscanProcessing.czi'
pos_ctrl_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_WT_1hrTor_10Pos_1_AiryscanProcessing.czi'
neg_empty_field_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_dFIP200_NoTreat_Empty_AiryscanProcessing.czi'
pos_empty_field_czi = '/Users/nweir/Dropbox/chris_imaging/imaging_test/th_sample_czi/lc3_p62_timecourse/LC3-p62_HEK_WT_1hrTor_Empty_AiryscanProcessing.czi'
svm_pkl = '/Users/nweir/Dropbox/code/csth-imaging/trained_svm.pkl' # change /path/to/ so it points to the csth-imaging folder

Now that you have defined the important variables, you will perform pre-processing. This takes the following steps:
1. Load images
2. Identify regions of the field containing cells
3. Identify nuclei
4. Segment cells  
    _Note:_ By default this segments cells using the 488 wavelength. Change 488 to your desired wavelength in the splitter.segment_cells() command to change wavelengths.
5. Initialize foci detection object.

The steps are indicated in the code blocks.

In [4]:
## NOTE: RUNNING

# 1. load images and the trained SVM focus classifier, and 2.
neg_ctrl_finder = find_cells.MultiFinder(filename=neg_ctrl_czi,
                                         bg_filename=neg_empty_field_czi,
                                         oof_svm=svm_pkl,
                                         optim=True)
pos_ctrl_finder = find_cells.MultiFinder(filename=pos_ctrl_czi,
                                         bg_filename=pos_empty_field_czi,
                                         oof_svm=svm_pkl,
                                         optim=True)
print('MultiFinder created.')

# 2. find cells
neg_ctrl_splitter = segment_cells.CellSplitter(neg_ctrl_finder)
pos_ctrl_splitter = segment_cells.CellSplitter(pos_ctrl_finder)

# 3. identify nuclei
neg_ctrl_splitter.segment_nuclei(verbose=True)
pos_ctrl_splitter.segment_nuclei(verbose=True)

# 4. Segment cells.
neg_ctrl_splitter.segment_cells(488, verbose=True)
pos_ctrl_splitter.segment_cells(488, verbose=True)

# 5. Initialize foci detection objects.
neg_foci = foci.Foci(neg_ctrl_splitter, verbose=True)
pos_foci = foci.Foci(pos_ctrl_splitter, verbose=True)

Optimization mode: only using first three images.
Optimization mode: only using first three images.
MultiFinder created.
generating cell masks...
log-transforming arrays...
applying gaussian filter...
computing p-value transformation...
 computing p-val xform for image 1 out of 3
 computing p-val xform for image 2 out of 3
 computing p-val xform for image 3 out of 3
converting to binary...

generating cell masks...

generating mask #1
labeling contiguous objects...
eliminating objects w/volume < 100,000 px...
pruning labels...
unlabeling out of focus slices...
generating gradient image...
generating gradient intensity histograms...
predicting focus labels using SVM classifier...
calculating decision function values for labels...
raw focus labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
correcting slice labels...
One continuous stretch defined as in focus.
appending outputs...
mask #1 complete.

generating mask #2
labeling

holes filled.
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
passing segmented objs and seeds to Nuclei instance...

segmenting image #2 of 3...
Using thresholds set based on slice intensity
thresholding floor set to 5264
slice-by-slice cutoffs:
[   179.5    165.     205.5    250.5    302.     340.5    349.     367.5
    448.     591.     883.5   1411.    2288.    3629.5   5350.5   7160.
   8515.    9227.5   9544.    9952.5  10121.   10801.5  12095.5  12811.5
  13162.   13063.5  12516.   11767.   11653.5  11583.5  11320.   10505.5
   9855.5   9587.5   9060.5   8309.5   7164.5   6002.5   4516.    3413.
   2569.    2009.    1500.    1122.     886.5    773.5    740.     671.
    609.     597.5    550.5]
generating threshold image...
raw image imported.
preprocessing complete.
filling holes in objects.
holes filled.
distance map complete.
smoothing distance m

/usr/local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


cell mask means:
[1732.8113001192426, 1793.4162702511028, 1855.3783902627229, 1915.9120247062222, 1972.4163322030129, 2023.6643205757302, 2069.5430382819991, 2110.5859807727761, 2147.595701905092, 2181.0876722561434]
slopes:
[61.28354507174015, 61.247877227559684, 58.518970970145006, 53.876147934754044, 48.563353039493109, 43.460830098522933, 39.02633181154647, 35.250845741683634]
slope deltas:
[ 0.03566784  2.72890626  4.64282304  5.3127949   5.10252294  4.43449829
  3.77548607]
desired erosions: 5
erosion of mask #0 complete.
eroding mask 1...
cell mask means:
[1448.4206456496045, 1502.6095819370344, 1557.1959634454324, 1609.4374516874741, 1657.1693481299933, 1699.9087619348409, 1738.0074961519108, 1772.4452368759273, 1804.0915214496536, 1833.4216545951406]
slopes:
[54.38765889791398, 53.413934875219866, 49.986692342280435, 45.235655123683387, 40.419074010958752, 36.268237470543227, 33.042012648871378, 30.48820885960663]
slope deltas:
[ 0.97372402  3.42724253  4.75103722  4.81658111 

Next, you're ready to try segmenting foci and checking the output to see if it looks correct.  
A bit of background on how Canny edge detection works so you can logically select values. There are two threshold numbers that go into detecting foci:
- high threshold: the minimum edge intensity to _seed_ a new object. The entire object doesn't have to have an edge this sharp, but if no part of it does, it won't be identified.
- low threshold: the minimum edge intensity to _grow_ an object. If one point already satisfied the high threshold, it can grow even if it has a dimmer edge as long as that dimmer edge is above the low threshold.  

If this doesn't make sense, read about the Canny edge detection algorithm on Wikipedia or elsewhere.

I often start with values of 10,000 and 7,500 for the high and low thresholds respectively, so those are the default values below.

Running the code block below will perform segmentation and save the raw images as well as the segmented foci to the output_dir you select in the first line. You can then open these images in fiji, overlay the segmented foci onto the raw images, and determine if the threshold is too high/too low. The code will also output some descriptive information about the number of dots, their intensities, etc. which may be useful to you in deciding thresholds.

In [5]:
from skimage import io  # for saving images
output_dir = '/Users/nweir/Dropbox/chris_imaging/timelapse_test/'  # change this to where you'd like the output images to go.
import os
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
# save raw images
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, pos_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'pos_raw_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.imgs[c][im, :, :, :])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, neg_foci.imgs[c].shape[0]):  # for each image in that channel
        io.imsave(output_dir + 'neg_raw_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.imgs[c][im, :, :, :])        

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_488_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_488_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_488_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_raw_561_1.tif is a low 

In [6]:
## RUNNING THIS BLOCK WILL TAKE A WHILE - BE PATIENT! ##
# edit the (high, low) threshold pairs below to optimize segmentation.
green_thresholds = (14000, 10500) 
red_thresholds = (9000, 7500)
# perform segmentation
neg_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
pos_foci.segment(verbose=True, thresholds={488: green_thresholds,
                                          561: red_thresholds})
# save segmentation outputs
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(pos_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'pos_foci_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.foci[c][im])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(neg_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'neg_foci_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.foci[c][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 488
------------------------------------------------------
canny threshold for channel 488: (14000, 10500)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 2011.78324413
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 19 foci in image
cell mean: 990.691041753
cell standard deviation: 1220.36429739
intensity cutoff for removal: 6482.33038001
-----filtering foci-----
focus intensity: 105.59483622
removing focus
focus intensity: 8045.1

focus intensity: 8825.95049505
focus intensity: 10930.3966942
focus intensity: 7370.44262295
focus intensity: 6971.81395349
focus intensity: 11737.2592593
focus intensity: 7048.15384615
focus intensity: 6178.55555556
removing focus
focus intensity: 9668.675
after eliminating dim foci: 16 foci in image
eliminating foci that reside outside of cells...
eliminating intranuclear foci...
15 final foci
--matching foci to parent cells--
current ID: 1
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[29]
matched to one parent cell.
current ID: 3
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[27]
matched to one parent cell.
current ID: 5
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[627]
matched to one parent cell.
current ID: 6
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[729]
matched to one parent cell.
current ID: 7
  overlapping parent cell(s): 
[2]
  # of pixels overla

focus intensity: 5073.41666667
focus intensity: 3716.98969072
removing focus
focus intensity: 5366.22839506
focus intensity: 4444.3
removing focus
focus intensity: 4914.8852459
focus intensity: 4154.25581395
removing focus
focus intensity: 4293.52173913
removing focus
focus intensity: 4368.69565217
removing focus
focus intensity: 4129.76666667
removing focus
focus intensity: 4915.67391304
focus intensity: 3844.25
removing focus
focus intensity: 5370.16129032
focus intensity: 4012.875
removing focus
focus intensity: 4433.0
removing focus
focus intensity: 4795.44047619
focus intensity: 5252.8
focus intensity: 3961.4
removing focus
focus intensity: 4564.5
removing focus
focus intensity: 4500.4
removing focus
focus intensity: 4411.18604651
removing focus
focus intensity: 3802.375
removing focus
focus intensity: 4742.7260274
focus intensity: 4001.2
removing focus
focus intensity: 3848.96875
removing focus
focus intensity: 6272.29824561
focus intensity: 5412.83333333
focus intensity: 5014.02

focus intensity: 4382.42857143
removing focus
focus intensity: 17601.1195361
focus intensity: 3412.4
removing focus
focus intensity: 4632.88421053
removing focus
focus intensity: 6406.27659574
focus intensity: 5352.04347826
focus intensity: 5412.65384615
focus intensity: 6472.25641026
focus intensity: 5132.16666667
focus intensity: 6007.0
focus intensity: 5317.57894737
focus intensity: 4873.8
focus intensity: 4923.92307692
focus intensity: 4359.2
removing focus
focus intensity: 5881.80519481
focus intensity: 4874.0
focus intensity: 4801.31111111
focus intensity: 5617.1
focus intensity: 5840.4893617
focus intensity: 4526.12121212
removing focus
focus intensity: 6261.24637681
focus intensity: 4557.4
removing focus
focus intensity: 5794.76027397
focus intensity: 5016.72413793
focus intensity: 6287.09722222
focus intensity: 4882.88888889
focus intensity: 5376.75
focus intensity: 3537.4
removing focus
focus intensity: 5286.61904762
focus intensity: 4074.85714286
removing focus
focus intensi

[1]
  # of pixels overlapping with parent(s):
[135]
matched to one parent cell.
current ID: 21
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[76]
matched to one parent cell.
current ID: 30
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[32]
matched to one parent cell.
current ID: 31
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[53]
matched to one parent cell.
current ID: 32
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[22]
matched to one parent cell.
current ID: 43
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[46]
matched to one parent cell.
current ID: 47
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[189]
matched to one parent cell.
current ID: 48
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[33]
matched to one parent cell.
current ID: 51
  overlapping parent cell(s):

...

  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[37]
matched to one parent cell.
current ID: 472
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[174]
matched to one parent cell.
current ID: 473
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[48]
matched to one parent cell.
current ID: 477
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[52]
matched to one parent cell.
current ID: 478
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[21]
matched to one parent cell.
current ID: 480
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[82]
matched to one parent cell.
current ID: 481
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[88]
matched to one parent cell.
current ID: 482
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[39]
matched to one parent cell.
current

  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[52]
matched to one parent cell.
current ID: 573
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[55]
matched to one parent cell.
current ID: 574
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[122]
matched to one parent cell.
current ID: 575
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[112]
matched to one parent cell.
current ID: 577
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[40]
matched to one parent cell.
current ID: 579
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[45]
matched to one parent cell.
current ID: 581
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[16]
matched to one parent cell.
current ID: 582
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[74]
matched to one parent cell.
curren

focus intensity: 4519.0
focus intensity: 4380.0
focus intensity: 5192.68292683
focus intensity: 4408.3
focus intensity: 5440.51162791
after eliminating dim foci: 81 foci in image
eliminating foci that reside outside of cells...
eliminating intranuclear foci...
79 final foci
--matching foci to parent cells--
current ID: 4
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[19]
matched to one parent cell.
current ID: 5
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[47]
matched to one parent cell.
current ID: 7
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[46]
matched to one parent cell.
current ID: 9
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[31]
matched to one parent cell.
current ID: 10
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[70]
matched to one parent cell.
current ID: 15
  overlapping parent cell(s): 
[1]
  # of pixels overla

  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[28]
matched to one parent cell.
current ID: 96
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[22]
matched to one parent cell.
current ID: 97
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[35]
matched to one parent cell.
current ID: 98
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[5]
matched to one parent cell.
current ID: 99
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[35]
matched to one parent cell.
current ID: 100
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[24]
matched to one parent cell.
current ID: 101
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[23]
matched to one parent cell.
current ID: 102
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[94]
matched to one parent cell.
current ID: 1

KeyboardInterrupt: 

In [ ]:
for c in pos_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(pos_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'pos_foci_' + str(c) + '_' + str(im) + '.tif',
                 pos_foci.foci[c][im])
for c in neg_foci.channels:  # for each channel
    if c == 405:
        continue
    for im in range(0, len(neg_foci.foci[c])):  # for each image in that channel
        io.imsave(output_dir + 'neg_foci_' + str(c) + '_' + str(im) + '.tif',
                 neg_foci.foci[c][im])

Edit the thresholds in the cell above and re-run as needed until you reach ideal thresholds.

In [15]:
%%notify
desired_channel = 561  # change this to 488 for green or 561 for red
# edit the (high, low) threshold pair below to optimize segmentation.
thresholds = (14000, 10500) 
# perform segmentation
neg_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
pos_foci.segment(verbose=True, thresholds={desired_channel: thresholds},
                 seg_channels=(desired_channel,))
# save segmentation outputs
for im in range(0, len(pos_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'pos_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             pos_foci.foci[desired_channel][im])
for im in range(0, len(neg_foci.foci[desired_channel])):  # for each image in that channel
    io.imsave(output_dir + 'neg_foci_' + str(desired_channel) + '_' + str(im) + '.tif',
             neg_foci.foci[desired_channel][im])

beginning segmentation.
------------------------------------------------------
segmenting foci from channel 561
------------------------------------------------------
canny threshold for channel 561: (14000, 10500)
segmenting foci for position 1 out of 3
generating normalized image for segmentation...
blurring image...
calculating image mean...
original cell mean: 456.103148621
normalizing image...
performing segmentation...
raw image imported.
performing gaussian filtering...
Image smoothed.
preprocessing complete.
generating distance map...
distance map complete.
smoothing distance map...
distance map smoothed.
identifying maxima...
maxima identified.
watershedding...
watershedding complete.
creating PexSegmentObj...
eliminating dim foci...
before eliminating dim foci: 18 foci in image
cell mean: 1064.94174767
cell standard deviation: 1218.31996163
intensity cutoff for removal: 4719.90163257
-----filtering foci-----
focus intensity: 145.954528607
removing focus
focus intensity: 6888.

focus intensity: 8531.78070175
focus intensity: 7317.85
focus intensity: 7626.28787879
focus intensity: 3869.66666667
removing focus
focus intensity: 6903.4
focus intensity: 9960.25581395
focus intensity: 10543.6384977
focus intensity: 11415.5069124
focus intensity: 5757.95833333
focus intensity: 8950.8125
focus intensity: 12029.9333333
focus intensity: 9999.38931298
focus intensity: 10652.0265487
focus intensity: 12633.127572
focus intensity: 7370.62962963
focus intensity: 8914.69354839
focus intensity: 7172.57142857
focus intensity: 8933.07692308
focus intensity: 7797.65909091
focus intensity: 9561.81012658
focus intensity: 12278.3956044
focus intensity: 7441.6
focus intensity: 10726.9411765
focus intensity: 11587.2222222
focus intensity: 7170.28571429
focus intensity: 7253.0990991
focus intensity: 7263.07407407
focus intensity: 6401.46987952
focus intensity: 9391.06338028
focus intensity: 5795.90909091
focus intensity: 6314.96491228
focus intensity: 12065.9306358
focus intensity: 59

current ID: 33
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[172]
matched to one parent cell.
current ID: 34
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[213]
matched to one parent cell.
current ID: 35
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[217]
matched to one parent cell.
current ID: 36
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[24]
matched to one parent cell.
current ID: 37
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[64]
matched to one parent cell.
current ID: 38
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[120]
matched to one parent cell.
current ID: 39
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[131]
matched to one parent cell.
current ID: 40
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[113]
matched to one parent 

current ID: 102
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[268]
matched to one parent cell.
current ID: 103
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[24]
matched to one parent cell.
current ID: 104
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[78]
matched to one parent cell.
current ID: 105
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[78]
matched to one parent cell.
current ID: 106
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[119]
matched to one parent cell.
current ID: 107
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[90]
matched to one parent cell.
current ID: 108
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[64]
matched to one parent cell.
current ID: 109
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[53]
matched to one par

  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[185]
matched to one parent cell.
current ID: 171
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[95]
matched to one parent cell.
current ID: 172
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[106]
matched to one parent cell.
current ID: 173
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[253]
matched to one parent cell.
current ID: 176
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[237]
matched to one parent cell.
current ID: 177
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[22]
matched to one parent cell.
current ID: 178
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[113]
matched to one parent cell.
foci segmented from position 1

segmenting foci for position 2 out of 3
generating normalized image for segmentation...
blurring 

...

current ID: 65
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[13]
matched to one parent cell.
current ID: 66
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[269]
matched to one parent cell.
current ID: 67
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[3]
matched to one parent cell.
current ID: 68
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[156]
matched to one parent cell.
current ID: 69
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[100]
matched to one parent cell.
current ID: 70
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[43]
matched to one parent cell.
current ID: 71
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[158]
matched to one parent cell.
current ID: 72
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[5]
matched to one parent cell

  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[28]
matched to one parent cell.
current ID: 132
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[22]
matched to one parent cell.
current ID: 133
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[118]
matched to one parent cell.
current ID: 134
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[83]
matched to one parent cell.
current ID: 135
  overlapping parent cell(s): 
[1]
  # of pixels overlapping with parent(s):
[56]
matched to one parent cell.
current ID: 136
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[197]
matched to one parent cell.
current ID: 137
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[176]
matched to one parent cell.
current ID: 138
  overlapping parent cell(s): 
[2]
  # of pixels overlapping with parent(s):
[33]
matched to one parent cell.
curre

/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_1.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/pos_foci_561_2.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/neg_foci_561_0.tif is a low contrast image
  warn('%s is a low contrast image' % fname)
/usr/local/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /Users/nweir/Dropbox/chris_imaging/timelapse_test/neg_foci_561_1.tif is a


------------------------------------------------------
segmentation complete for channel 561
------------------------------------------------------


<IPython.core.display.Javascript object>

'/Users/nweir/Dropbox/chris_imaging/timelapse_test/'